# Exercises for the course "Declarative Problem Solving Paradigms in AI"

In [1]:
# !pip install cpmpy numpy pychoco gurobipy networkx colorama tqdm hyperopt --quiet

import cpmpy as cp
import numpy as np

You can ignore google* and tensor* dependency errors.

## **Session 5 B: CP search strategies and algorithm configuration**

This is the second part (B) to the fifth exercise session of Declarative Problem Solving Paradigms in AI.

This notebook covers part B:
- **A**: Chapter 6: Solving technologies and encodings
- **B**: Chapter 7: CP search strategies and algorithm configuration

**Useful Resources:**
* CPMpy documentation: https://cpmpy.readthedocs.io/en/latest/index.html
* CPMpy quickstart: https://cpmpy.readthedocs.io/en/latest/modeling.html
* List of supported solvers / solving technologies: https://cpmpy.readthedocs.io/en/latest/#supported-solvers
* Documentation on hyperparameter configuration: https://cpmpy.readthedocs.io/en/latest/modeling.html#hyperparameter-search-across-different-parameters

### **Part 1: Algorithm Selection**

Depending on the (type of) problem that you're trying to solve, one solver backend / solving technology might be more suited than the other. It is always a good idea to experiment a bit as to see what works best, before settling on one particular backend (and possibly start making optimisations tailored specifically towards that one solver).

Below is a collection of models you've worked on throughout the previous exercises:

- Task allocation problem
- Team assignment problem
- Pizza voucher problem

Simply run the 3 cells and continue below...

In [2]:
def task_allocation():
    """See exercise session 2"""

    # Parameters of the problem
    n_tasks = 6        # Number of tasks
    n_workers = 6      # Number of workers

    # Decision Variables
    task_assignment = None # task_assignment[j] == i iff worker i is assigned to task j
    task_assignment = cp.intvar(0, n_workers-1, shape=n_tasks, name="task_assignment") # task_assignment[j] = the worker assigned to task j

    # Model
    model = cp.Model()

    # Constraints
    model += []
    # 1) Each task is assigned to exactly one worker.
    # -> no need to model this constraint, since the chosen encoding for 'task_assignment'
    #    will already enforce this (each task j can only have one worker task_assignment[j] as value)
    # 2) Allow each worker to maximally have one task assigned = no to tasks can have the same worker
    model += cp.AllDifferent(task_assignment)

    # Conditional implications
    model += (task_assignment[0] == 0).implies(task_assignment[2] != 1) # If worker 0 is assigned to task 0, worker 1 not on task 2
    model += (task_assignment[1] == 1).implies(task_assignment[0] == 2) # If worker 1 is assigned to task 1, worker 2 must be assigned to task 0
    model += (task_assignment[2] == 2).implies(task_assignment[1] != 0) # If worker 2 is assigned to task 2, worker 0 cannot be on task 1

    return model


In [3]:
def team_skill_assignment():
    """See exercise session 2"""

    # Parameters of the Problem
    E = 25   # number of employees
    P = 10   # number of projects
    S = 10   # number of skills

    np.random.seed(0)  # for reproducibility

    # Skill matrix: employee_skills[e][s] is 1 if employee e has skill s, else 0
    employee_skills = np.random.randint(0, 2, (E, S))

    # Project skill requirements: project_skills[p][s] is 1 if project p requires skill s, else 0
    project_skills = np.random.randint(0, 2, (P, S))

    # Team size constraints
    T_min = 2  # minimum number of employees in a team
    T_max = 5  # maximum number of employees in a team

    # Decision Variables
    # - team_assign[p, e]: 1 if employee e is assigned to project p, else 0
    team_assign = cp.boolvar(shape=(P, E), name="team_assign")

    # Model
    model = cp.Model()

    # Constraints
    for p in range(P):
        # 1) Each team should have between T_min and T_max employees
        model += T_min <= cp.sum(team_assign[p, :])
        model += cp.sum(team_assign[p, :]) <= T_max

        for s in range(S):
            # 2) Skill requirement for each project
            if project_skills[p, s]: # if project p requires skill s
                # (count the number of employees which have skill s and are assigned to team p)
                model += ( cp.sum(team_assign[p, e] for e in range(E) if employee_skills[e, s]) > 0 )


    # 3) Employees can only be assigned to at most one project
    model += ( np.sum(team_assign, axis=0) <= 1 )

    return model

In [4]:
def pizza_voucher():
    """See exercise session 3"""

    # Parameters of the problem
    prices = [50, 60, 90, 70, 80, 100, 20, 30, 40, 10]  # prices of pizzas
    vouchers = [
        (1, 2), (2, 3), (1, 1), (0, 1), (2, 1),
        (2, 2), (3, 3), (1, 0), (3, 2)]  # available vouchers

    nPizzas = len(prices)
    nVouchers = len(vouchers)

    # Decision variables:
    # what voucher is used for each pizza, negative means the pizza has to be paid,
    # positive means it is free. 0 means no voucher is used
    # take care, this means we are using 'prices' and 'vouchers' with indexes starting from 1, and not 0.
    v = cp.intvar(-nVouchers, nVouchers, shape=nPizzas)

    # define variables that keep track of how many free and paid pizza's each voucher has.
    p = cp.intvar(0, nPizzas, shape=nVouchers)
    f = cp.intvar(0, nPizzas, shape=nVouchers)

    # model the constraints:
    pizza_model = cp.Model()

    # number of paid and free pizza's in f and p must correspond to the voucher data in v.
    pizza_model += [p[i] == cp.Count(v, -(i + 1)) for i in range(nVouchers)]
    pizza_model += [f[i] == cp.Count(v, i + 1) for i in range(nVouchers)]
    # voucher can only have free pizza's on it if it has enough paid pizza's
    pizza_model += [(f[i] > 0).implies(p[i] == vouchers[i][0]) for i in range(nVouchers)]
    # a voucher can not have too many free pizza's on it
    pizza_model += [f[i] <= vouchers[i][1] for i in range(nVouchers)]
    # free pizza's must be cheaper than paid pizza's
    pizza_model += [(v[i] > 0).implies(v[i] != -v[j]) for i in range(nPizzas) for j in range(nPizzas) if
                    prices[j] < prices[i]]

    # minimize the total price
    pizza_model.minimize(sum([(v[i] <= 0) * prices[i] for i in range(nPizzas)]))
    return pizza_model

Now write some code to test each of these problems on a set of solvers and print the resulting runtimes. You can get the runtime from a model/solver by using `.status().runtime`. To more clearly see the differences, you may manually fill in the below table (also possible on paper).

| Problem | OR-Tools | Choco | Gurobi |
| - | - | - | - |
| Task allocation | ... | ... | ... |
| Team skill assignment | ... | ... | ... |
| Pizza voucher | ... | ... | ... |


In [5]:
?cp.Model.solve

Signature: cp.Model.solve(self, solver=None, time_limit=None, **kwargs)

Docstring:

Send the model to a solver and get the result


:param solver: name of a solver to use. Run SolverLookup.solvernames() to find out the valid solver names on your system. (default: None = first available solver)

:type string: None (default) or a name in SolverLookup.solvernames() or a SolverInterface class (Class, not object!)


:param time_limit: optional, time limit in seconds

:type time_limit: int or float


:return: Bool: the computed output:

- True      if a solution is found (not necessarily optimal, e.g. could be after timeout)

- False     if no solution is found

In [6]:
from cpmpy import SolverLookup
import pandas as pd

def algorithm_selection(models: dict[str, cp.Model], solvers: list) -> pd.DataFrame:
    """
    TODO: Given a collection of problem models and a list of solvers,
          run each problem on each solver and print the measured runtimes.

          'models' is a dictionary of 'problem name' -> 'problem CPMpy model' mappings.

    HINT: To immediately print (for instant feedback), use 'flush=True'
              as an argument, i.e. print(..., flush=True).

    Running this code will take some time. Place a time limit of 30s on each solve call.
    """
    results = {}
    for solver in solvers:
        results[solver] = {}
        for problem, model in models.items():
            model.solve(solver=solver, time_limit=30)
            results[solver][problem] = model.status().runtime
    return pd.DataFrame(results)

solvers = ["ortools", "choco", "gurobi"]
models = {
    "task_allocation": task_allocation(),
    "team_skill_assignment": team_skill_assignment(),
    "pizza_voucher": pizza_voucher(),
}
algorithm_selection(models, solvers)


Restricted license - for non-production use only - expires 2026-11-23


,ortools,choco,gurobi
task_allocation,0.004772,0.004304,0.001699
team_skill_assignment,0.009228,0.003215,0.000098
pizza_voucher,0.093473,6.483276,3.682045


**Observations**:

1) Which solver would you recommend?

2) Would that recommendation apply to all problems, to all instances of a problem?


### **Part 2: Algorithm Configuration**

Once we've made a decision on a particular solving backend that we want to target, we can start looking at configuring said backend. Each solver (OR-Tools, Choco, Exact, ...) has is own unique interface. And thus each solver exposes their own unique set of options that we can start tweaking in order to get more performance out of it. By default, solvers will often use some heuristic as to guess what a good set of options could be. In order to stay performant, not much time on this optimization will be spend. We on the other hand can do our own experiments and figure out which set of settings work best for the particular problem that we're trying to solve.

This happens often in industry. Imagine that a company has a fleet of vehicles and wants to use CP as to optimize their driving paths. Once a model has been made and a solver has been chosen, they can start fine-tuning said solver for that particular type of problem that they'll have to solve over and over again.

#### **0. Setup**

In [7]:
import time
from cpmpy.solvers.choco import CPM_choco
from cpmpy.solvers.solver_interface import SolverInterface, SolverStatus, ExitStatus
from cpmpy.expressions.variables import _NumVarImpl, _IntVarImpl, _BoolVarImpl

class strategy_CPM_Choco(CPM_choco):
    """A slightly adapted Choco solver class as to support changing the search strategy."""

    def solve(self, time_limit=None, strategy="default", **kwargs):
        """
            Call the Choco solver

            Arguments:
            - time_limit:  maximum solve time in seconds (float, optional)
            - kwargs:      any keyword argument, sets parameters of solver object

        """
        # ensure all vars are known to solver
        self.solver_vars(list(self.user_vars))

        # call the solver, with parameters
        self.chc_solver = self.chc_model.get_solver()

        # Get all variables in the model as to configure the search strategy for them
        chc_vars = self.solver_vars(list(self.user_vars))

        # Setting search strategy
        if strategy=="default":
            pass
        elif strategy=="input-order-lb-value":
            self.chc_solver.set_input_order_lb_search(chc_vars)
        elif strategy=="random":
            self.chc_solver.set_random_search(chc_vars)
        elif strategy=="domain-size-over-weighted-degree":
            self.chc_solver.set_dom_over_w_deg_search(chc_vars)
        elif strategy=="activity":
            self.chc_solver.set_activity_based_search(chc_vars)
        elif strategy=="conflict-history":
            self.chc_solver.set_conflict_history_search(chc_vars)
        elif strategy=="failure-rate":
            self.chc_solver.set_failure_rate_based_search(chc_vars)
        elif strategy=="pick-on-domain":
            self.chc_solver.set_pick_on_dom_search(chc_vars)
        elif strategy=="smallest-domain-lb-value":
            self.chc_solver.set_min_dom_lb_search(chc_vars)
        elif strategy=="smallest-domain-ub-value":
            self.chc_solver.set_min_dom_ub_search(chc_vars)
        else:
            raise ValueError(f"Invalid strategy: {strategy}")

        # Make call to parent class to perform actual solve operation
        return super().solve(**kwargs)

#### **1. Search Strategy**

Below is a CP model for the "Magic sequence problem". As described in CSPLib:

"A magic sequence of length $n$ is a sequence of integers $x_0...x_{n−1}$ between $0$ and $n−1$, such that for all $i$
in $0$ to $n−1$, the number $i$ occurs exactly $x_i$ times in the sequence. For instance, $\{6,2,1,0,0,0,1,0,0,0\}$ is a magic sequence since $0$ occurs $6$ times in it, $1$ occurs twice, etc."

It is not all too important to understand this problem / the below model.

In [8]:
import numpy as np

def magic_sequence(n):
    """Returns a model for the "Magic Sequence" problem"""

    # Decision Variables
    x = cp.intvar(0, n-1, shape=n, name="x")

    # Model
    model = cp.Model()

    # Constraints
    # 1) number i occurs x[i] times
    for i in range(n):
        model += x[i] == sum(x == i)

    # 2) redundant constraints to speed up search
    model += sum(x) == n
    model += sum(x * np.array([i for i in range(n)])) == n

    return model, x

"""Example usage of the model for a problem instance of size 10."""
# Create model of size 10
model, x = magic_sequence(n=10)

# Create Choco solver
#   Custom Choco solver object that acts exactly the same as
#   when created using SolverLookup().get("choco", model),
#   besides allowing you to pass a search strategy when
#   calling '.solve()'
solver = strategy_CPM_Choco(model)

# Solve
solver.solve(strategy="default") # Can also leave this blank, as "default" is the default

# Show results
print(solver.status())
print(x.value())

ExitStatus.OPTIMAL (0.0006840229034423828 seconds)
[6 2 1 0 0 0 1 0 0 0]


Using an instance of the new solver class `strategy_CPM_Choco` from the setup section, evaluate the below set of strategies for a problem instance of size `n=50`. See the above example on how to use `strategy_CPM_Choco`.

Generate a similar table as seen in the lectures:

| Strategy name | Seconds |
| - | - |
| default | ... |
| input-order-lb-value | ... |
| ... | ... |

In [9]:
# The different search strategies for the Choco solver
STRATEGIES = [
    "default",
    "input-order-lb-value",
    "random",
    "domain-size-over-weighted-degree",
    "activity",
    "conflict-history",
    "failure-rate",
    "pick-on-domain",
    "smallest-domain-lb-value",
    "smallest-domain-ub-value",
]

In [10]:
def make_magic_table(n: int = 50) -> pd.DataFrame:
    """n: problem size"""

    # Create model
    model, x = magic_sequence(n)

    """
    TODO: Write code to create the above table, one row for each strategy.

    HINT: A new solver class is available for this exercise.
          Instead of using SolverLookup, create the solver instance using
          "solver = strategy_CPM_Choco(model)". The `.solve()` method of this
          object accepts a new argument: strategy. Just pass it the name of
          the strategy and Choco will be configured accordingly.

    WARNING: Do not re-use the solver object between the strategies.
            Re-using the solver object will activate its incremental mode,
            i.e. searching for a new solution from where the last solve call left off.
            In this case, we actually want to compare the strategies against each other
            in the exact same setting and thus always solve 'from scratch'.
            => Always re-create the solver instance between experiments.

    HINT: You can use `pandas` to create nice tables.
          More specifically:
            import pandas as pd
            pd.DataFrame(...)

          To get the table:
            | a | b |
            | - | - |
            | 1 | 2 |
            | 3 | 4 |
          You would need to call
            pd.DataFrame([       # a list of rows
              {'a': 1, 'b': 2},  # row = dictionary of column values
              {'a': 3, 'b': 4}
            ])
    """

    import pandas as pd

    # ┌────SOLUTION────┐
    res = []  # to collect the rows of the table

    for strategy in STRATEGIES:
        solver = strategy_CPM_Choco(model)
        solver.solve(strategy=strategy)

        res.append({"Strategy": strategy, "Seconds": solver.status().runtime})

    df = pd.DataFrame(res)
    df.sort_values(by="Seconds", inplace=True)  # sort to easily see which strategy is the best
    # └────────────────┘

    return df


make_magic_table(n=50)

,Strategy,Seconds
9,smallest-domain-ub-value,0.004488
6,failure-rate,0.005908
4,activity,0.006653
3,domain-size-over-weighted-degree,0.007350
8,smallest-domain-lb-value,0.009428
2,random,0.009796
7,pick-on-domain,0.011586
1,input-order-lb-value,0.011636
0,default,0.023125
5,conflict-history,0.060628


**Observations**:

Try for different values of `n`. What do you observe?

Which search strategy is best depends not only on the type of problem, but can also on the particular instance. This motivates the research domain of using Machine Learning techniques to predict the best strategy based on features of the problem/instance.

### **Part 3: Hyperparameter Configuration**

When the number of configurable solver-options becomes too big or the model becomes expensive to evaluate, testing all possible combinations becomes infeasible. We only have the resources to test a subset of them. But which subset should we test and how can we do this in a smart way as to come as close as possible to the ideal set of parameters (for the specific problem on which we're fine-tuning the solver)?

We can interpret the "algorithm configuration problem" as a search problem. All possible parameter combinations define our search space. Now we want to efficiently traverse this space in search for the most optimal combinations within a limited number of iterations.

In the lectures we saw three such approaches:
- Grid search
- Random search
- Model-based search

Let's fine-tune the OR-Tools solver on the N-Queens problem:

In [11]:
def nqueens(N):
    """A model for the N-queens problem; i.e. how to position N queens on a
    NxN chessboard such that no two queens attack each other."""
    # Variables (one per row)
    queens = cp.intvar(1, N, shape=N, name="queens")

    # Constraints on columns and left/right diagonal
    m = cp.Model([
        cp.AllDifferent(queens),
        cp.AllDifferent([queens[i] + i for i in range(N)]),
        cp.AllDifferent([queens[i] - i for i in range(N)]),
    ])

    return (m, queens)

In [12]:
# The search space of solver options
#   If you want to try more, you can look here: https://github.com/CPMpy/cpmpy/blob/4494396b2302110c3740ac46164208c25e77266a/cpmpy/solvers/ortools.py#L580
space = {
    'cp_model_probing_level': [0, 1, 2, 3],
    'linearization_level': [0, 1, 2], # -> not seen in lecture slides
    'symmetry_level': [0, 1, 2],
    'search_branching': [0, 1, 2],
    'use_phase_saving': [False, True],
}

#### **Grid search**

With grid search, we discretize our search-space and simply go over each possible combination one by one (our search space is already discrete). Once the budget on the number of iterations has been exceeded or all combinations have been tested (whichever comes first), we halt and return the best parameters we've found.

Complete the code for grid search.

In [13]:
import itertools

def grid_search(model, solver: str, space, N: int = 10):
    """
    TODO: Go over each possible parameter combination inside the search space one by one.
          Create a solver object from the model and the provided solver name.
          Solve with the selected parameters and collect the runtime.
          After all combinations have been tested or the iteration limit N has been reached,
          return the best set of parameters found so far.

    HINT: s.status().runtime gets you the runtime of a solver object.

    HINT: To use a set of parameter which are given as a dictionary, use the following syntax:
          s.solve(**chosen_parameters).

    BONUS: To speed up the search, add "Adaptive Capping". Simply set the time_limit of the solve
           call to the best runtime found so far (no need continue searching for a solution when the solve
           call already took longer then the best we found so far).
    """

    best_runtime = float('inf')
    best_parameters = None

    # Create all possible combinations from the search space
    combinations = list(itertools.product(*space.values()))

    # Iterate over all possible combinations
    for i, chosen_parameters in enumerate(combinations):

        # IMPORTANT: Max number of iterations
        if i >= N:
            break

        # Get parameters as a dictionary: param name -> param value
        chosen_parameters = {k:v for (k,v) in zip(space.keys(), chosen_parameters)}

        """
        TODO: Complete code as described above
        """
        s = SolverLookup().get(solver, model)
        s.solve(time_limit=best_runtime, **chosen_parameters)
        if runtime := s.status().runtime:
            best_runtime, best_parameters = runtime, chosen_parameters

        """
        TODO: Print the measured runtime.

        HINT: To immediately print (for instant feedback), use 'flush=True'
              as an argument, i.e. print(..., flush=True).
        """
        # ┌────SOLUTION────┐
        if s.status().runtime == best_runtime: # Some bonus pretty printing to cleary see when search found a better set of parameters
            print(chosen_parameters, "->", '\033[1m [', s.status().runtime, '] \033[0m', flush=True)
        else:
            print(chosen_parameters, "->", s.status().runtime, flush=True)
    
    # Return best set of parameters
    return {k:v for k,v in zip(space.keys(), best_parameters)}


# Generate model
model, x = nqueens(100)
# Perform configuration search
grid_params = grid_search(model, "ortools", space, N=10)
# Print found configuration
print("BEST:", grid_params)

{'cp_model_probing_level': 0, 'linearization_level': 0, 'symmetry_level': 0, 'search_branching': 0, 'use_phase_saving': False} ->  [ 0.10698400000000001 ] 
{'cp_model_probing_level': 0, 'linearization_level': 0, 'symmetry_level': 0, 'search_branching': 0, 'use_phase_saving': True} ->  [ 0.11551 ] 
{'cp_model_probing_level': 0, 'linearization_level': 0, 'symmetry_level': 0, 'search_branching': 1, 'use_phase_saving': False} ->  [ 0.11880600000000001 ] 
{'cp_model_probing_level': 0, 'linearization_level': 0, 'symmetry_level': 0, 'search_branching': 1, 'use_phase_saving': True} ->  [ 0.117399 ] 
{'cp_model_probing_level': 0, 'linearization_level': 0, 'symmetry_level': 0, 'search_branching': 2, 'use_phase_saving': False} ->  [ 0.106393 ] 
{'cp_model_probing_level': 0, 'linearization_level': 0, 'symmetry_level': 0, 'search_branching': 2, 'use_phase_saving': True} ->  [ 0.11677100000000001 ] 
{'cp_model_probing_level': 0, 'linearization_level': 0, 'symmetry_level': 1, 'search_branching': 0, '

**Q**: What do you observe? Do you believe that this approach has a good chance of finding the best set of parameters? What if we increased the number of iterations `N`?

Grid search systematically goes over the entire search space. When the number of iterations is limited, only a very local section of the search space will be tested. The chance that the best set of parameters is located there, is rather small. Even more so, there is no guarantee that there are any good configurations located in that small section; some parameters are never changed. Even when we increase the number of iterations `N`, grid search will still stay rather local and might miss all good configuration.

#### **Random Search**

Instead of systematic evaluation, going over the configurations one by one in order, implement a random configuration search for a fixed number of `N` solve calls.

Complete the code for random search.

In [14]:
import random
import tqdm

random.seed(42)

def random_search(model, solver: str, space, N:int=10):
    """
    TODO: In each iteration, randomly select a new parameter combination (one that has not been tested before).
          Create a solver object from the model based and the provided solver name.
          Solve with the selected parameters and collect the runtime.
          After all combinations have been tested or the iteration limit N has been reached,
          return the best set of parameters found so far.

    HINT: s.status().runtime gets you the runtime of a solver object.

    HINT: To use a set of parameter which are given as a dictionary, use the following syntax:
          s.solve(**chosen_parameters).

    HINT: Instead of randomly sampling N configurations from the set of all configurations,
          you could also randomly shuffle that set and then use the exact same code as
          for grid search to test the first N 'random' configurations.
          You can use 'random.shuffle()' to shuffle a list (this happens in-place,
          so the original list gets updated).

    BONUS: To speed up the search, add "Adaptive Capping". Simply set the time_limit of the solve
           call to the best runtime found so far (no need continue searching for a solution when the solve
           call already took longer then the best we found so far).
    """

    best_runtime = float('inf')
    best_parameters = None

    # Create all possible combinations from the search space
    combinations = list(itertools.product(*space.values()))

    """
    TODO: Randomly shuffle the search space as to later get N random configurations.
    """
    random.shuffle(combinations)

    # Iterate over all possible combinations
    for i, chosen_parameters in enumerate(combinations):

        # IMPORTANT: Max number of iterations
        if i >= N:
            break

        # Get parameters as a dictionary: param name -> param value
        chosen_parameters = {k:v for (k,v) in zip(space.keys(), chosen_parameters)}

        """
        TODO: Complete code as described above
        """
        s = SolverLookup().get(solver, model)
        s.solve(time_limit=best_runtime, **chosen_parameters)
        if runtime := s.status().runtime:
            best_runtime, best_parameters = runtime, chosen_parameters

        """
        TODO: Print the measured runtime.

        HINT: To immediately print (for instant feedback), use 'flush=True'
              as an argument, i.e. print(..., flush=True).
        """
        # ┌────SOLUTION────┐
        if s.status().runtime == best_runtime: # Some bonus pretty printing to cleary see when search found a better set of parameters
            print(chosen_parameters, "->", '\033[1m [', s.status().runtime, '] \033[0m', flush=True)
        else:
            print(chosen_parameters, "->", s.status().runtime, flush=True)
        # └────────────────┘


    # Return best set of parameters
    return {k:v for k,v in zip(space.keys(), best_parameters)}


# Generate model
model, x = nqueens(100)
# Perform configuration search
random_params = random_search(model, "ortools", space, 10)
# Print found configuration
print(random_params)


{'cp_model_probing_level': 0, 'linearization_level': 2, 'symmetry_level': 0, 'search_branching': 0, 'use_phase_saving': False} ->  [ 0.142685 ] 
{'cp_model_probing_level': 2, 'linearization_level': 2, 'symmetry_level': 0, 'search_branching': 1, 'use_phase_saving': False} ->  [ 0.142869 ] 
{'cp_model_probing_level': 0, 'linearization_level': 1, 'symmetry_level': 2, 'search_branching': 0, 'use_phase_saving': False} ->  [ 0.15547 ] 
{'cp_model_probing_level': 1, 'linearization_level': 1, 'symmetry_level': 1, 'search_branching': 0, 'use_phase_saving': True} ->  [ 0.169776 ] 
{'cp_model_probing_level': 2, 'linearization_level': 1, 'symmetry_level': 1, 'search_branching': 0, 'use_phase_saving': True} ->  [ 0.16263 ] 
{'cp_model_probing_level': 0, 'linearization_level': 2, 'symmetry_level': 1, 'search_branching': 1, 'use_phase_saving': True} ->  [ 0.18507200000000001 ] 
{'cp_model_probing_level': 1, 'linearization_level': 1, 'symmetry_level': 0, 'search_branching': 0, 'use_phase_saving': True

Run the above code a few times.

**Observations**:

1) How do the tested configurations of random search compare to those of grid search?

2) Do you think that this approach has a better chance of finding a good set of parameters within the limited number of iterations? Why?

By randomly sampling from the search space, our samples will be more diverse. More configuration parameters will be experimented with, ones that the gridsearch has never gotten around to change. Theoretically, this approach should increase the probability of finding good combinations.

Still, there is no guarantee that random search will be better than grid search. If grid search starts in a good region it might already give a good result. If it starts in a bad area, it might give a bad result.

Model-based search adds some logic as to how to select the most promising region(s) from the search space.

#### **Model-based Search**

As a last option, we'll use HyperOpt's model-based approach. Here, a probabilistic model gets made during the search, which is utilised to guide the search towards regions of high "promise".

Complete the code for the model-based approach.

In [15]:
from hyperopt import hp, tpe, fmin, Trials

# Convert the search-space to a hyperopt format
h_space = {k:hp.choice(k,v) for k,v in space.items()}
best_runtime = float("inf")

# Create the model
model, x = nqueens(100)

def objective(model, solver: str, parameters):
    """Objective function to guide HyperOpt during its search.

    TODO: Create a solver object, run the solver with the given params, and return the runtime.

    Hint: Put a limit on the solve time (e.g. 20s) as to not wait too long.
    """
    global best_runtime

    s = SolverLookup().get(solver, model)
    s.solve(**parameters, time_limit=20)
    runtime = s.status().runtime
    
    """
    TODO: Print the measured runtime.

    HINT: To immediately print (for instant feedback), use 'flush=True'
          as an argument, i.e. print(..., flush=True).
    """
    # ┌────SOLUTION────┐
    if runtime < best_runtime: # not necessary, is used for prettier printing
        best_runtime = runtime
    if runtime == best_runtime: # not necessary, is used for prettier printing
        print(parameters, "->", '\033[1m [', runtime, '] \033[0m')

    else:
        print(parameters, "->", runtime)
    # └────────────────┘

    return runtime


NUM_ITERATIONS = 10  # <- limit on the number of iterations N (increase to e.g. 30)

# Run HyperOpt's model-based algorithm configuration.
model_params = fmin(
    fn=lambda p: objective(model, 'ortools', p),
    space=h_space,
    algo=tpe.suggest,
    trials=Trials(),
    max_evals=NUM_ITERATIONS,
    show_progressbar=False
)
# Print found configuration
print(model_params)

{'cp_model_probing_level': 1, 'linearization_level': 1, 'search_branching': 0, 'symmetry_level': 2, 'use_phase_saving': True} ->  [ 0.394618 ] 
{'cp_model_probing_level': 3, 'linearization_level': 2, 'search_branching': 2, 'symmetry_level': 1, 'use_phase_saving': True} -> 0.40509
{'cp_model_probing_level': 1, 'linearization_level': 1, 'search_branching': 2, 'symmetry_level': 0, 'use_phase_saving': True} ->  [ 0.316024 ] 
{'cp_model_probing_level': 0, 'linearization_level': 0, 'search_branching': 0, 'symmetry_level': 0, 'use_phase_saving': True} ->  [ 0.098291 ] 
{'cp_model_probing_level': 2, 'linearization_level': 0, 'search_branching': 1, 'symmetry_level': 0, 'use_phase_saving': False} -> 1.097725
{'cp_model_probing_level': 0, 'linearization_level': 2, 'search_branching': 1, 'symmetry_level': 0, 'use_phase_saving': True} -> 0.6646380000000001
{'cp_model_probing_level': 3, 'linearization_level': 0, 'search_branching': 0, 'symmetry_level': 2, 'use_phase_saving': True} -> 0.4009310000000

**Observations**:

1) How well does the model-based approach work compared to the other approaches?

2) Increase the number of iterations, what do you see?

With a very limited number of iterations, the model-based approach will not realise its full potential. It will start with some random samples to try and identify some patterns/correlations (exploration), but will not be able to build its probabilitic model in time to start benefitting from it (exploitation). So it should be quite similar to the random search approach. By increasing the number of iterations, we should start seeing it search more systrematically; staying close to known good regions in the search space and preventing regions with very bad regions.